<a href="https://colab.research.google.com/github/DouglasMiyagui/E-Commerce_KDD/blob/main/E-Commerce_Pr%C3%A9-Processamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<center>
  <font color=red size=10>Pré-Processamento</font>
</center>


<font size=5>Esta etapa é essencial, pois estabelece as bases para toda a análise subsequente. Será realizado o processo de limpeza de dados, tratamento de valores ausentes, codificação de variáveis categóricas e normalização dos dados, assegurando que o conjunto de dados esteja preparado e de alta qualidade para análises subsequentes. O objetivo é criar um ambiente de dados confiável e consistente para explorar e modelar.</font>

# <font color=red>1 - Provisionamento</font>

## 1.1 - Instalar e importar dependências

In [3]:
!pip install pandas==2.0.1
!pip install unidecode


[notice] A new release of pip available: 22.2.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 3.1 MB/s eta 0:00:0000:01

[notice] A new release of pip available: 22.2.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from unidecode import unidecode
from sklearn.preprocessing import LabelEncoder

In [ ]:
print('Versão do pandas -> %s' % pd.__version__)
print('Versão do numpy -> %s' % np.__version__)
print('Versão do matplotlib -> %s' % plt.matplotlib.__version__)
print('Versão do csv -> %s' % csv.__version__)
print('Versão do unidecode -> 1.3.6')

Versão do pandas -> 2.0.1
Versão do numpy -> 1.23.5
Versão do matplotlib -> 3.7.1
Versão do csv -> 1.0
Versão do unidecode -> 1.3.6


## 1.2 - Carregamentos dos arquivos

In [39]:
geolocalizacao = pd.read_csv('../data/dados_brutos/olist_geolocation_dataset.csv')
clientes = pd.read_csv('../data/dados_brutos/olist_customers_dataset.csv')
vendedores = pd.read_csv('../data/dados_brutos/olist_sellers_dataset.csv')
produtos = pd.read_csv('../data/dados_brutos/olist_products_dataset.csv')
itens_pedidos = pd.read_csv('../data/dados_brutos/olist_order_items_dataset.csv')
pedidos = pd.read_csv('../data/dados_brutos/olist_orders_dataset.csv')
pagamentos = pd.read_csv('../data/dados_brutos/olist_order_payments_dataset.csv')
avaliacoes = pd.read_csv('../data/dados_brutos/olist_order_reviews_dataset.csv')

## 1.3 - Criar os mapeamentos que serão utilizados em mais de um DataFrame

### 1.3.1 - Mapeamento dos CEPs, das latitudes e longitudes para serem inseridas nos DataFrames clientes e vendedores.

In [40]:
mapeamento_geolocalizacao = dict(zip(geolocalizacao['geolocation_zip_code_prefix'],
                                     zip(geolocalizacao['geolocation_lat'],
                                         geolocalizacao['geolocation_lng'])))

### 1.3.2 - Mapeamento do número de referência das cidades, salvar em arquivo csv, para transformar os dados qualitativos em numerais nos dataframes clientes e vendedores para posteriormente usar nos modelos de machine learning.

In [41]:
# Tratando a inconsistência de um dado antes de gerar o mapeamento
vendedores.loc[vendedores['seller_city'] == '04482255', 'seller_city'] = 'recreio dos bandeirantes'

In [42]:
# Tratamento das inconsistências possíveis nas strings de cidades dos DataFrames clientes e vendedores
clientes['customer_city'] = clientes['customer_city'].apply(lambda x: unidecode(x).replace('ç', 'c').lower().strip())
vendedores['seller_city'] = vendedores['seller_city'].apply(lambda x: unidecode(x).replace('ç', 'c').lower().strip())

In [43]:
# Extrair cidades únicas dos DataFrames "clientes" e "vendedores"
cidades_clientes = clientes['customer_city'].unique()
cidades_vendedores = vendedores['seller_city'].unique()

# Unir as listas de cidades únicas e remover duplicatas
cidades_unicas = list(set().union(cidades_clientes, cidades_vendedores))

# Ordenar as cidades em ordem alfabética
cidades_unicas = sorted(cidades_unicas)

# Criar um mapeamento de cidades com números de referência a partir do número 1
mapeamento_cidades = {cidade: idx + 1 for idx, cidade in enumerate(cidades_unicas)}

# Criar um DataFrame com o mapeamento
df_mapeamento_cidades = pd.DataFrame(mapeamento_cidades.items(), columns=['cidade', 'referencia_cidade'])

# Salvar o mapeamento em um arquivo CSV
df_mapeamento_cidades.to_csv('../data/dados_tratados/mapeamento_cidades.csv', index=False)

In [44]:
df_mapeamento_cidades

,cidade,referencia_cidade
0,abadia de goias,1
1,abadia dos dourados,2
2,abadiania,3
3,abaete,4
4,abaetetuba,5
...,...,...
4189,xinguara,4190
4190,xique-xique,4191
4191,zacarias,4192
4192,ze doca,4193


### 1.3.3 - Mapeamento do número de referência dos estados, salvar em arquivo csv, para transformar os dados qualitativos em numerais nos dataframes clientes e vendedores para posteriormente usar nos modelos de machine learning.

In [45]:
# Tratamento das inconsistências possíveis nas strings de estados dos DataFrames clientes e vendedores
clientes['customer_state'] = clientes['customer_state'].str.upper().str.strip()
vendedores['seller_state'] = vendedores['seller_state'].str.upper().str.strip()

In [46]:
# Extrair estados únicos dos DataFrames "clientes" e "vendedores"
estados_clientes = clientes['customer_state'].unique()
estados_vendedores = vendedores['seller_state'].unique()

# Unir as listas de estados únicos e remover duplicatas
estados_unicos = list(set().union(estados_clientes, estados_vendedores))

# Ordenar os estados em ordem alfabética
estados_unicos = sorted(estados_unicos)

# Criar um mapeamento de estados com números de referência a partir do número 1
mapeamento_estados = {estado: idx + 1 for idx, estado in enumerate(estados_unicos)}

# Criar um DataFrame com o mapeamento
df_mapeamento_estados = pd.DataFrame(mapeamento_estados.items(), columns=['estado', 'referencia_estado'])

# Salvar o mapeamento em um arquivo CSV
df_mapeamento_estados.to_csv('../data/dados_tratados/mapeamento_estados.csv', index=False)

In [47]:
df_mapeamento_estados

,estado,referencia_estado
0,AC,1
1,AL,2
2,AM,3
3,AP,4
4,BA,5
5,CE,6
6,DF,7
7,ES,8
8,GO,9
9,MA,10


# <font color=red>2 - Clientes</font>

## 2.1 - Renomear as colunas

In [48]:
lista_colunas = clientes.columns.tolist()
print(lista_colunas)

['customer_id', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']


In [49]:
clientes.rename(columns={
    'customer_id': 'cliente_id',
    'customer_unique_id': 'cliente_id_unico',
    'customer_zip_code_prefix': 'cliente_cep',
    'customer_city': 'cliente_cidade',
    'customer_state': 'cliente_estado'
}, inplace=True)

## 2.2 - Criar as colunas com as geolocalizações para futuramente poder plotar mapas de localização dos clientes

In [50]:
clientes['cliente_lat'], clientes['cliente_lon'] = zip(*clientes['cliente_cep'].apply(lambda cep: mapeamento_geolocalizacao.get(cep, (None, None))))

## 2.3 - Visão geral do conjunto de dados

In [51]:
clientes

,cliente_id,cliente_id_unico,cliente_cep,cliente_cidade,cliente_estado,cliente_lat,cliente_lon
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,-20.500253,-47.400367
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,-23.724844,-46.549350
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,-23.531294,-46.656404
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,-23.493944,-46.180817
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,-22.993053,-47.139272
...,...,...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP,-23.584362,-46.497362
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP,-23.614720,-46.779405
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE,-3.734408,-38.507778
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS,-29.955478,-51.169328


In [52]:
print(clientes.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   cliente_id        99441 non-null  object 
 1   cliente_id_unico  99441 non-null  object 
 2   cliente_cep       99441 non-null  int64  
 3   cliente_cidade    99441 non-null  object 
 4   cliente_estado    99441 non-null  object 
 5   cliente_lat       99163 non-null  float64
 6   cliente_lon       99163 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 5.3+ MB
None


Valores únicos em cada coluna:

In [53]:
clientes.nunique()

cliente_id          99441
cliente_id_unico    96096
cliente_cep         14994
cliente_cidade       4119
cliente_estado         27
cliente_lat         14833
cliente_lon         14833
dtype: int64

Valores duplicados:

In [54]:
clientes.duplicated().sum()

0

Valores nulos em cada coluna:

In [55]:
clientes.isna().sum()

cliente_id            0
cliente_id_unico      0
cliente_cep           0
cliente_cidade        0
cliente_estado        0
cliente_lat         278
cliente_lon         278
dtype: int64

## 2.4 - Lidando com valores ausentes nas colunas de latitudes e longitudes

Para tratar os dados nulos de latitude e longitude, será colocado os dados da mesma cidade ignorando a exatidão de localização do CEP.

In [56]:
# Criar um DataFrame com informações únicas de cidade, latitude e longitude
cidades_info = clientes[['cliente_cidade', 'cliente_lat', 'cliente_lon']].drop_duplicates()

# Remover cidades duplicadas aleatoriamente para manter apenas uma referência única para cada cidade
cidades_info = cidades_info.sample(frac=1).drop_duplicates(subset='cliente_cidade')

# Criar um dicionário de mapeamento de cidades com latitudes e longitudes
mapeamento_cidades = dict(zip(cidades_info['cliente_cidade'], zip(cidades_info['cliente_lat'], cidades_info['cliente_lon'])))

# Preencher valores nulos nas colunas "cliente_lat" e "cliente_lon" usando o mapeamento de cidades
clientes['cliente_lat'] = clientes['cliente_lat'].fillna(clientes['cliente_cidade'].map(lambda cidade: mapeamento_cidades.get(cidade, (None, None))[0]))
clientes['cliente_lon'] = clientes['cliente_lon'].fillna(clientes['cliente_cidade'].map(lambda cidade: mapeamento_cidades.get(cidade, (None, None))[1]))

In [57]:
clientes.isna().sum()

cliente_id           0
cliente_id_unico     0
cliente_cep          0
cliente_cidade       0
cliente_estado       0
cliente_lat         61
cliente_lon         61
dtype: int64

Após fazer o tratamento dos dados nulos inserindo latitudes e longitudes referente a cidade, dos 278 dados nulos foi identificado 60 registros sem referência específica para suas cidades. Para evitar a perda desses dados em visualizações gráficas, será usada coordenadas aleatórias correspondentes aos seus respectivos estados.

In [58]:
# Criar um DataFrame com informações únicas de estado, latitude e longitude
estados_info = clientes[['cliente_estado', 'cliente_lat', 'cliente_lon']].drop_duplicates()

# Remover estados duplicados aleatoriamente para manter apenas uma referência única para cada estado
estados_info = estados_info.sample(frac=1).drop_duplicates(subset='cliente_estado')

# Criar um dicionário de mapeamento de estados com latitudes e longitudes
mapeamento_estados = dict(zip(estados_info['cliente_estado'], zip(estados_info['cliente_lat'], estados_info['cliente_lon'])))

# Preencher valores nulos nas colunas "cliente_lat" e "cliente_lon" usando o mapeamento de estados
clientes['cliente_lat'] = clientes['cliente_lat'].fillna(clientes['cliente_estado'].map(lambda estado: mapeamento_estados.get(estado, (None, None))[0]))
clientes['cliente_lon'] = clientes['cliente_lon'].fillna(clientes['cliente_estado'].map(lambda estado: mapeamento_estados.get(estado, (None, None))[1]))


In [59]:
clientes.isna().sum()

cliente_id          0
cliente_id_unico    0
cliente_cep         0
cliente_cidade      0
cliente_estado      0
cliente_lat         0
cliente_lon         0
dtype: int64

## 2.5 - Tratamento das strings dos atributos qualitativos nominais.

### 2.5.1 - Coluna cliente_cidade

In [60]:
clientes['cliente_cidade'] = clientes['cliente_cidade'].apply(lambda x: unidecode(x).replace('ç', 'c').lower().strip())

### 2.5.2 - Coluna cliente_estado

In [61]:
clientes['cliente_estado'] = clientes['cliente_estado'].str.upper().str.strip()

## 2.6 - Criar a coluna "cliente_regiao" para analises futuras

In [63]:
estado_para_regiao = {
    'AC': 'norte',
    'AL': 'nordeste',
    'AP': 'norte',
    'AM': 'norte',
    'BA': 'nordeste',
    'CE': 'nordeste',
    'DF': 'centro-oeste',
    'ES': 'sudeste',
    'GO': 'centro-oeste',
    'MA': 'nordeste',
    'MT': 'centro-oeste',
    'MS': 'centro-oeste',
    'MG': 'sudeste',
    'PA': 'norte',
    'PB': 'nordeste',
    'PR': 'sul',
    'PE': 'nordeste',
    'PI': 'nordeste',
    'RJ': 'sudeste',
    'RN': 'nordeste',
    'RS': 'sul',
    'RO': 'norte',
    'RR': 'norte',
    'SC': 'sul',
    'SP': 'sudeste',
    'SE': 'nordeste',
    'TO': 'norte'
}

In [64]:
clientes['cliente_regiao'] = clientes['cliente_estado'].map(estado_para_regiao)

## 2.7 - Codificação de Variáveis Categóricas em Numéricos

Após fazer o carregamento do mapeamento das cidades, será criada a coluna cliente_cidade_ref com o número de referência da cidade para poder usar esse atributo nos modelos de machine learning.

In [65]:
# Carregar o mapeamento de cidades a partir do arquivo CSV
mapeamento_cidades = pd.read_csv('../data/dados_tratados/mapeamento_cidades.csv')

# Criar um dicionário a partir do mapeamento carregado
mapeamento_final_cidades = dict(zip(mapeamento_cidades['cidade'], mapeamento_cidades['referencia_cidade']))

# Aplicar o mapeamento no DataFrame "clientes"
clientes['cliente_cidade_ref'] = clientes['cliente_cidade'].map(mapeamento_final_cidades)

Após carregar o mapeamento dos estados, será criada a coluna cliente_estado_ref com o número de referência do estado para poder usar esse atributo nos modelos de machine learning.

In [66]:
# Carregar o mapeamento de estados a partir do arquivo CSV
mapeamento_estados = pd.read_csv('../data/dados_tratados/mapeamento_estados.csv')

# Criar um dicionário a partir do mapeamento carregado
mapeamento_final_estados = dict(zip(mapeamento_estados['estado'], mapeamento_estados['referencia_estado']))

# Aplicar o mapeamento no DataFrame "clientes"
clientes['cliente_estado_ref'] = clientes['cliente_estado'].map(mapeamento_final_estados)

Será criada a coluna cliente_regiao_ref com o número de referência da região para poder usar esse atributo nos modelos de machine learning.

In [67]:
regioes = clientes['cliente_regiao']
lencoder = LabelEncoder()
regioes_numeros = lencoder.fit_transform(regioes)
clientes['cliente_regiao_ref'] = regioes_numeros

In [69]:
# Salvar o mapeamento em um arquivo CSV
mapeamento_df = pd.DataFrame({'cliente_regiao': regioes, 'cliente_regiao_ref': regioes_numeros})
mapeamento_df.to_csv('../data/dados_tratados/mapeamento_regioes.csv', index=False)

## 2.8 - Ordenando as colunas para melhor compreensão

In [70]:
lista_colunas = clientes.columns.tolist()
print(lista_colunas)

['cliente_id', 'cliente_id_unico', 'cliente_cep', 'cliente_cidade', 'cliente_estado', 'cliente_lat', 'cliente_lon', 'cliente_regiao', 'cliente_cidade_ref', 'cliente_estado_ref', 'cliente_regiao_ref']


In [71]:
nova_ordem_colunas = ['cliente_id', 'cliente_id_unico', 'cliente_cep', 'cliente_cidade', 'cliente_cidade_ref', 'cliente_estado', 'cliente_estado_ref', 'cliente_regiao', 'cliente_regiao_ref', 'cliente_lat', 'cliente_lon']
clientes = clientes[nova_ordem_colunas]

## 2.9 - DataFrame clientes após tratamento

In [72]:
clientes.head(20)

,cliente_id,cliente_id_unico,cliente_cep,cliente_cidade,cliente_cidade_ref,cliente_estado,cliente_estado_ref,cliente_regiao,cliente_regiao_ref,cliente_lat,cliente_lon
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,1405,SP,26,sudeste,3,-20.500253,-47.400367
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,3484,SP,26,sudeste,3,-23.724844,-46.549350
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,3658,SP,26,sudeste,3,-23.531294,-46.656404
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,2375,SP,26,sudeste,3,-23.493944,-46.180817
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,721,SP,26,sudeste,3,-22.993053,-47.139272
5,879864dab9bc3047522c92c82e1212b8,4c93744516667ad3b8f1fb645a3116a4,89254,jaragua do sul,1963,SC,24,sul,4,-26.453239,-49.117683
6,fd826e7cf63160e536e0908c76c3f441,addec96d2e059c80c30fe6871d30d177,4534,sao paulo,3658,SP,26,sudeste,3,-23.583194,-46.672290
7,5e274e7a0c3809e14aba7ad5aae0d407,57b2a98a409812fe9618067b6b8ebe4f,35182,timoteo,3939,MG,11,sudeste,3,-19.580580,-42.642151
8,5adf08e34b2e993982a47070956c5c65,1175e95fb47ddff9de6b2b06188f7e0d,81560,curitiba,1163,PR,18,sul,4,-25.477518,-49.224184
9,4b7139f34592b3a31687243a302fa75b,9afe194fb833f79e300e37e580171f22,30575,belo horizonte,465,MG,11,sudeste,3,-19.966590,-43.970939


In [73]:
clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   cliente_id          99441 non-null  object 
 1   cliente_id_unico    99441 non-null  object 
 2   cliente_cep         99441 non-null  int64  
 3   cliente_cidade      99441 non-null  object 
 4   cliente_cidade_ref  99441 non-null  int64  
 5   cliente_estado      99441 non-null  object 
 6   cliente_estado_ref  99441 non-null  int64  
 7   cliente_regiao      99441 non-null  object 
 8   cliente_regiao_ref  99441 non-null  int64  
 9   cliente_lat         99441 non-null  float64
 10  cliente_lon         99441 non-null  float64
dtypes: float64(2), int64(4), object(5)
memory usage: 8.3+ MB


## 2.10 - Salvar DataFrame Clientes Tratado

In [74]:
clientes.to_csv('../data/dados_tratados/clientes.csv', index=False)

# <font color=red>3 - Vendedores</font>

## 3.1 - Renomear as colunas

In [ ]:
lista_colunas = vendedores.columns.tolist()
print(lista_colunas)

['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state']


In [ ]:
vendedores.rename(columns={
    'seller_id': 'vendedor_id',
    'seller_zip_code_prefix': 'vendedor_cep',
    'seller_city': 'vendedor_cidade',
    'seller_state': 'vendedor_estado'
}, inplace=True)

## 3.2 - Criar as colunas com as geolocalizações para futuramente poder plotar mapas de localização dos vendedores

In [ ]:
vendedores['vendedor_lat'], vendedores['vendedor_lon'] = zip(*vendedores['vendedor_cep'].apply(lambda cep: mapeamento_geolocalizacao.get(cep, (None, None))))

## 3.3 - Visão geral do conjunto de dados

In [ ]:
vendedores


,vendedor_id,vendedor_cep,vendedor_cidade,vendedor_estado,vendedor_lat,vendedor_lon
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP,-22.899471,-47.064267
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP,-22.374632,-46.941224
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ,-22.907863,-43.175693
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP,-23.656354,-46.610917
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP,-22.958423,-46.534450
...,...,...,...,...,...,...
3090,98dddbc4601dd4443ca174359b237166,87111,sarandi,PR,-23.452487,-51.867759
3091,f8201cab383e484733266d1906e2fdfa,88137,palhoca,SC,-27.622133,-48.675013
3092,74871d19219c7d518d0090283e03c137,4650,sao paulo,SP,-23.660173,-46.678758
3093,e603cf3fec55f8697c9059638d6c8eb5,96080,pelotas,RS,-31.748636,-52.310751


In [ ]:
print(vendedores.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   vendedor_id      3095 non-null   object 
 1   vendedor_cep     3095 non-null   int64  
 2   vendedor_cidade  3095 non-null   object 
 3   vendedor_estado  3095 non-null   object 
 4   vendedor_lat     3088 non-null   float64
 5   vendedor_lon     3088 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 145.2+ KB
None


Valores únicos em cada coluna:

In [ ]:
print(vendedores.nunique())

vendedor_id        3095
vendedor_cep       2246
vendedor_cidade     609
vendedor_estado      23
vendedor_lat       2239
vendedor_lon       2239
dtype: int64


Valores duplicados:

In [ ]:
vendedores.duplicated().sum()

0

Valores nulos em cada coluna:

In [ ]:
vendedores.isnull().sum()

vendedor_id        0
vendedor_cep       0
vendedor_cidade    0
vendedor_estado    0
vendedor_lat       7
vendedor_lon       7
dtype: int64

## 3.4 - Lidando com valores ausentes nas colunas de latitudes e longitudes

Para tratar os dados nulos de latitude e longitude, será colocado os dados da mesma cidade ignorando a exatidão de localização do CEP.

In [ ]:
# Criar um DataFrame com informações únicas de cidade, latitude e longitude
cidades_info = vendedores[['vendedor_cidade', 'vendedor_lat', 'vendedor_lon']].drop_duplicates()

# Remover cidades duplicadas aleatoriamente para manter apenas uma referência única para cada cidade
cidades_info = cidades_info.sample(frac=1).drop_duplicates(subset='vendedor_cidade')

# Criar um dicionário de mapeamento de cidades com latitudes e longitudes
mapeamento_cidades_vendedores = dict(zip(cidades_info['vendedor_cidade'], zip(cidades_info['vendedor_lat'], cidades_info['vendedor_lon'])))

In [ ]:
# Preencher valores nulos nas colunas "vendedor_lat" e "vendedor_lon" usando o mapeamento de cidades dos vendedores
vendedores['vendedor_lat'] = vendedores['vendedor_lat'].fillna(vendedores['vendedor_cidade'].map(lambda cidade: mapeamento_cidades_vendedores.get(cidade, (None, None))[0]))
vendedores['vendedor_lon'] = vendedores['vendedor_lon'].fillna(vendedores['vendedor_cidade'].map(lambda cidade: mapeamento_cidades_vendedores.get(cidade, (None, None))[0]))


In [ ]:
vendedores.isnull().sum()

vendedor_id        0
vendedor_cep       0
vendedor_cidade    0
vendedor_estado    0
vendedor_lat       0
vendedor_lon       0
dtype: int64

Após fazer o tratamento dos dados nulos inserindo latitudes e longetudes referente a cidade, dos 7 dados nulos foi identificado 1 registros sem referência específica para suas cidades. Para evitar a perda desses dados em visualizações gráficas, será usado coordenadas aleatórias correspondentes aos seus respectivos estados.

In [ ]:
# Criar um DataFrame com informações únicas de estado, latitude e longitude
estados_info = vendedores[['vendedor_estado', 'vendedor_lat', 'vendedor_lon']].drop_duplicates()

# Remover estados duplicados aleatoriamente para manter apenas uma referência única para cada estado
estados_info = estados_info.sample(frac=1).drop_duplicates(subset='vendedor_estado')

# Criar um dicionário de mapeamento de estados com latitudes e longitudes
mapeamento_estados_vendedores = dict(zip(estados_info['vendedor_estado'], zip(estados_info['vendedor_lat'], estados_info['vendedor_lon'])))

# Preencher valores nulos nas colunas "cliente_lat" e "cliente_lon" usando o mapeamento de estados
vendedores['vendedor_lat'] = vendedores['vendedor_lat'].fillna(vendedores['vendedor_estado'].map(lambda estado: mapeamento_estados_vendedores.get(estado, (None, None))[0]))
vendedores['vendedor_lon'] = vendedores['vendedor_lon'].fillna(vendedores['vendedor_estado'].map(lambda estado: mapeamento_estados_vendedores.get(estado, (None, None))[1]))

In [ ]:
vendedores.isnull().sum()

vendedor_id        0
vendedor_cep       0
vendedor_cidade    0
vendedor_estado    0
vendedor_lat       0
vendedor_lon       0
dtype: int64

## 3.5 - Criar a coluna "vendedor_regiao" para futuras análises

Será usado o mapeamento das regiões feito no tópico 2.6 (criar a coluna cliente_regiao)

In [ ]:
vendedores['vendedor_regiao'] = vendedores['vendedor_estado'].map(estado_para_regiao)

## 3.6 - Codificação de Variáveis Categóricas em Numéricos

Após carregar o mapeamento das cidades, será criada a coluna "vendedor_cidade_ref" com o número de referência da cidade para poder usar esse atributo nos modelos de machine learning.

In [ ]:
# Carregar o mapeamento de cidades a partir do arquivo CSV
mapeamento_cidades = pd.read_csv('mapeamento_cidades.csv')

# Criar um dicionário a partir do mapeamento carregado
mapeamento_final_cidades = dict(zip(mapeamento_cidades['cidade'], mapeamento_cidades['referencia_cidade']))

# Aplicar o mapeamento no DataFrame "vendedores"
vendedores['vendedor_cidade_ref'] = vendedores['vendedor_cidade'].map(mapeamento_final_cidades)

Após carregar o mapeamento dos estados, será criada a coluna "vendedor_estado_ref" com o número de referência do estado para poder usar esse atributo nos modelos de machine learning.

In [ ]:
# Carregar o mapeamento de estados a partir do arquivo CSV
mapeamento_estados = pd.read_csv('mapeamento_estados.csv')

# Criar um dicionário a partir do mapeamento carregado
mapeamento_final_estados = dict(zip(mapeamento_estados['estado'], mapeamento_estados['referencia_estado']))

# Aplicar o mapeamento no DataFrame "vendedores"
vendedores['vendedor_estado_ref'] = vendedores['vendedor_estado'].map(mapeamento_final_estados)

Será criada a coluna "vendedor_regiao_ref" com o número de referência da região para poder usar esse atributo nos modelos de machine learning.

In [ ]:
regioes = vendedores['vendedor_regiao']
lencoder = LabelEncoder()
regioes_numeros = lencoder.fit_transform(regioes)
vendedores['vendedor_regiao_ref'] = regioes_numeros

## 3.7 - Ordenando as colunas para melhor compreensão

In [ ]:
lista_colunas = vendedores.columns.tolist()
print(lista_colunas)

['vendedor_id', 'vendedor_cep', 'vendedor_cidade', 'vendedor_estado', 'vendedor_lat', 'vendedor_lon', 'vendedor_regiao', 'vendedor_cidade_ref', 'vendedor_estado_ref', 'vendedor_regiao_ref']


In [ ]:
nova_ordem_colunas = ['vendedor_id', 'vendedor_cep', 'vendedor_cidade', 'vendedor_cidade_ref', 'vendedor_estado', 'vendedor_estado_ref', 'vendedor_regiao', 'vendedor_regiao_ref', 'vendedor_lat', 'vendedor_lon']
vendedores = vendedores[nova_ordem_colunas]

## 3.8 - DataFrame Vendedores após tratamento

In [ ]:
vendedores.head()

,vendedor_id,vendedor_cep,vendedor_cidade,vendedor_cidade_ref,vendedor_estado,vendedor_estado_ref,vendedor_regiao,vendedor_regiao_ref,vendedor_lat,vendedor_lon
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,721,SP,26,sudeste,3,-22.899471,-47.064267
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,2377,SP,26,sudeste,3,-22.374632,-46.941224
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,3200,RJ,19,sudeste,3,-22.907863,-43.175693
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,3658,SP,26,sudeste,3,-23.656354,-46.610917
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,564,SP,26,sudeste,3,-22.958423,-46.534450


In [ ]:
vendedores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   vendedor_id          3095 non-null   object 
 1   vendedor_cep         3095 non-null   int64  
 2   vendedor_cidade      3095 non-null   object 
 3   vendedor_cidade_ref  3095 non-null   int64  
 4   vendedor_estado      3095 non-null   object 
 5   vendedor_estado_ref  3095 non-null   int64  
 6   vendedor_regiao      3095 non-null   object 
 7   vendedor_regiao_ref  3095 non-null   int64  
 8   vendedor_lat         3095 non-null   float64
 9   vendedor_lon         3095 non-null   float64
dtypes: float64(2), int64(4), object(4)
memory usage: 241.9+ KB


# <font color=red>4 - Produtos</font>

## 4.1 - Renomear as colunas

In [ ]:
lista_colunas = produtos.columns.tolist()
print(lista_colunas)

['product_id', 'product_category_name', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']


In [ ]:
produtos.rename(columns={
    'product_id': 'produto_id',
    'product_category_name': 'produto_categoria',
    'product_name_lenght': 'comprimento_nome_produto',
    'product_description_lenght': 'comprimento_descricao_produto',
    'product_photos_qty': 'quantidade_fotos_produto',
    'product_weight_g': 'peso_produto_gramas',
    'product_length_cm': 'comprimento_produto_cm',
    'product_height_cm': 'altura_produto_cm',
    'product_width_cm': 'largura_produto_cm'
}, inplace=True)

## 4.2 - Visão geral do conjunto de dados

In [ ]:
produtos

,produto_id,produto_categoria,comprimento_nome_produto,comprimento_descricao_produto,quantidade_fotos_produto,peso_produto_gramas,comprimento_produto_cm,altura_produto_cm,largura_produto_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0
...,...,...,...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,45.0,67.0,2.0,12300.0,40.0,40.0,40.0
32947,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,41.0,971.0,1.0,1700.0,16.0,19.0,16.0
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,50.0,799.0,1.0,1400.0,27.0,7.0,27.0
32949,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0


In [ ]:
print(produtos.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   produto_id                     32951 non-null  object 
 1   produto_categoria              32341 non-null  object 
 2   comprimento_nome_produto       32341 non-null  float64
 3   comprimento_descricao_produto  32341 non-null  float64
 4   quantidade_fotos_produto       32341 non-null  float64
 5   peso_produto_gramas            32949 non-null  float64
 6   comprimento_produto_cm         32949 non-null  float64
 7   altura_produto_cm              32949 non-null  float64
 8   largura_produto_cm             32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB
None


Valores únicos em cada coluna:

In [ ]:
produtos.nunique()

produto_id                       32951
produto_categoria                   73
comprimento_nome_produto            66
comprimento_descricao_produto     2960
quantidade_fotos_produto            19
peso_produto_gramas               2204
comprimento_produto_cm              99
altura_produto_cm                  102
largura_produto_cm                  95
dtype: int64

Valores duplicados:

In [ ]:
produtos.duplicated().sum()

0

Valores nulos em cada coluna:

In [ ]:
print(produtos.isnull().sum())

produto_id                         0
produto_categoria                610
comprimento_nome_produto         610
comprimento_descricao_produto    610
quantidade_fotos_produto         610
peso_produto_gramas                2
comprimento_produto_cm             2
altura_produto_cm                  2
largura_produto_cm                 2
dtype: int64


## 4.3 - Lidando com valores ausentes

Não é conveniente apagar as linhas com dados nulos porque iria perder muitos dados das vendas feitas, como não há a possibilidade de recuperar esses dados e existem apenas 610 dados nulos em um lista de 32.949 lançamentos, a coluna categoria do produto será preenchida com o valor "indefinido" e os outros atributos após usá-los para criar a coluna produto serão excluídas, pois não serão relevantes nas análises e nos modelos.

In [ ]:
produtos['produto_categoria'].fillna("indefinido", inplace=True)

Será preenchido os restantes dos valores nulos das outras colunas por "0" para poder criar o atributo "produto" em seguida.

In [ ]:
produtos.fillna(0, inplace=True)

Será criado o atributo "produto" para poder usá-lo nas análises, para isso será levado em consideração que um produto que tenha os atributos ['produto_categoria', 'comprimento_nome_produto', 'comprimento_descricao_produto', 'quantidade_fotos_produto', 'peso_produto_gramas', 'comprimento_produto_cm', 'altura_produto_cm', 'largura_produto_cm'] com os mesmos valores é consequência de ser o mesmo produto, por isso, a coluna produto será criada com o valor da categoria do produto e a soma de todas as outras colunas.

In [ ]:
produtos['produto'] = produtos['produto_categoria'] + ' ' + (
    produtos['comprimento_nome_produto'] +
    produtos['comprimento_descricao_produto'] +
    produtos['quantidade_fotos_produto'] +
    produtos['peso_produto_gramas'] +
    produtos['comprimento_produto_cm'] +
    produtos['altura_produto_cm'] +
    produtos['largura_produto_cm']
).astype(int).astype(str)

## 4.4 - Remover as colunas irrelevantes

Agora que já foram usados os dados das colunas de pesos e medidas para identificar os produtos iguais, serão removidas as colunas que não serão usadas para as análises e para os modelos de machine learning.

In [ ]:
colunas_remover = ['comprimento_nome_produto', 'comprimento_descricao_produto', 'quantidade_fotos_produto', 'peso_produto_gramas', 'comprimento_produto_cm', 'altura_produto_cm', 'largura_produto_cm']
produtos = produtos.drop(columns=colunas_remover)

## 4.5 - Padronizar as categorias para uma melhor análise

In [ ]:
categorias_unicas = produtos['produto_categoria'].unique().tolist()
categorias_unicas

['perfumaria',
 'artes',
 'esporte_lazer',
 'bebes',
 'utilidades_domesticas',
 'instrumentos_musicais',
 'cool_stuff',
 'moveis_decoracao',
 'eletrodomesticos',
 'brinquedos',
 'cama_mesa_banho',
 'construcao_ferramentas_seguranca',
 'informatica_acessorios',
 'beleza_saude',
 'malas_acessorios',
 'ferramentas_jardim',
 'moveis_escritorio',
 'automotivo',
 'eletronicos',
 'fashion_calcados',
 'telefonia',
 'papelaria',
 'fashion_bolsas_e_acessorios',
 'pcs',
 'casa_construcao',
 'relogios_presentes',
 'construcao_ferramentas_construcao',
 'pet_shop',
 'eletroportateis',
 'agro_industria_e_comercio',
 'indefinido',
 'moveis_sala',
 'sinalizacao_e_seguranca',
 'climatizacao',
 'consoles_games',
 'livros_interesse_geral',
 'construcao_ferramentas_ferramentas',
 'fashion_underwear_e_moda_praia',
 'fashion_roupa_masculina',
 'moveis_cozinha_area_de_servico_jantar_e_jardim',
 'industria_comercio_e_negocios',
 'telefonia_fixa',
 'construcao_ferramentas_iluminacao',
 'livros_tecnicos',
 'elet

In [ ]:
mapeamento_categorias = {
    'agro_industria_e_comercio': 'industria_comercio',
    'alimentos': 'alimentos_bebidas',
    'alimentos_bebidas': 'alimentos_bebidas',
    'artes': 'artes_artesanatos',
    'artes_e_artesanato': 'artes_artesanatos',
    'artigos_de_festas': 'casa_decoracao',
    'artigos_de_natal': 'casa_decoracao',
    'audio': 'eletronicos',
    'automotivo': 'automotivo',
    'bebes': 'bebes',
    'bebidas': 'alimentos_bebidas',
    'beleza_saude': 'beleza_perfumaria',
    'brinquedos': 'brinquedos',
    'cama_mesa_banho': 'casa_decoracao',
    'casa_conforto': 'casa_decoracao',
    'casa_conforto_2': 'casa_decoracao',
    'casa_construcao': 'construcao',
    'cds_dvds_musicais': 'filmes_musica',
    'cine_foto': 'eletronicos',
    'climatizacao': 'eletrodomesticos',
    'consoles_games': 'games',
    'construcao_ferramentas_construcao': 'construcao',
    'construcao_ferramentas_ferramentas': 'construcao',
    'construcao_ferramentas_iluminacao': 'construcao',
    'construcao_ferramentas_jardim': 'construcao',
    'construcao_ferramentas_seguranca': 'construcao',
    'cool_stuff': 'casa_decoracao',
    'dvds_blu_ray': 'eletronicos',
    'eletrodomesticos': 'eletrodomesticos',
    'eletrodomesticos_2': 'eletrodomesticos',
    'eletronicos': 'eletronicos',
    'eletroportateis': 'eletronicos',
    'esporte_lazer': 'esporte_lazer',
    'fashion_bolsas_e_acessorios': 'bolsas_malas',
    'fashion_calcados': 'roupas_calcados',
    'fashion_esporte': 'roupas_calcados',
    'fashion_roupa_feminina': 'roupas_calcados',
    'fashion_roupa_infanto_juvenil': 'roupas_calcados',
    'fashion_roupa_masculina': 'roupas_calcados',
    'fashion_underwear_e_moda_praia': 'roupas_calcados',
    'ferramentas_jardim': 'jardinagem',
    'flores': 'jardinagem',
    'fraldas_higiene': 'bebes',
    'indefinido': 'outros',
    'industria_comercio_e_negocios': 'industria_comercio',
    'informatica_acessorios': 'informatica',
    'instrumentos_musicais': 'filmes_musica',
    'la_cuisine': 'casa_decoracao',
    'livros_importados': 'livros',
    'livros_interesse_geral': 'livros',
    'livros_tecnicos': 'livros',
    'malas_acessorios': 'bolsas_malas',
    'market_place': 'outros',
    'moveis_colchao_e_estofado': 'moveis',
    'moveis_cozinha_area_de_servico_jantar_e_jardim': 'moveis',
    'moveis_decoracao': 'moveis',
    'moveis_escritorio': 'moveis',
    'moveis_quarto': 'moveis',
    'moveis_sala': 'moveis',
    'musica': 'filmes_musica',
    'papelaria': 'papelaria',
    'pc_gamer': 'games',
    'pcs': 'informatica',
    'perfumaria': 'beleza_perfumaria',
    'pet_shop': 'pet_shop',
    'portateis_casa_forno_e_cafe': 'eletrodomesticos',
    'portateis_cozinha_e_preparadores_de_alimentos': 'eletrodomesticos',
    'relogios_presentes': 'relogios',
    'seguros_e_servicos': 'industria_comercio',
    'sinalizacao_e_seguranca': 'industria_comercio',
    'tablets_impressao_imagem': 'informatica',
    'telefonia': 'telefonia',
    'telefonia_fixa': 'telefonia',
    'utilidades_domesticas': 'casa_decoracao'
}
produtos['produto_categoria'] = produtos['produto_categoria'].map(mapeamento_categorias)

## 4.6 - Codificação da variável "produto_categoria" em numéricos

A coluna produtos não será convertida em numeral porque não será usado nos modelos de machine learning pela falta de exatidão do dado, então, só será convertida a coluna produto_categoria.

In [ ]:
categorias = produtos['produto_categoria']
lencoder = LabelEncoder()
categorias_numeros = lencoder.fit_transform(categorias)
produtos['produto_categoria_ref'] = categorias_numeros

## 4.7 - DataFrame produtos após tratamento

In [ ]:
produtos.head()

,produto_id,produto_categoria,produto,produto_categoria_ref
0,1e9e8ef04dbcff4541ed26657ea517e5,beleza_perfumaria,perfumaria 593,4
1,3aa071139cb16b67ca9e5dea641aaa2f,artes_artesanatos,artes 1389,1
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,esporte_lazer 493,11
3,cef67bcfe19066a932b7673e239eb23d,bebes,bebes 716,3
4,9dc1a7de274444849c219cff195d0b71,casa_decoracao,utilidades_domesticas 1118,7


In [ ]:
produtos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   produto_id             32951 non-null  object
 1   produto_categoria      32951 non-null  object
 2   produto                32951 non-null  object
 3   produto_categoria_ref  32951 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 1.0+ MB


# <font color=red>5 - Itens_Pedidos</font>

## 5.1 - Renomear as colunas

In [ ]:
itens_pedidos.rename(columns={
    'order_id': 'pedido_id',
    'order_item_id': 'quantidade',
    'product_id': 'produto_id',
    'seller_id': 'vendedor_id',
    'shipping_limit_date': 'data_limite_envio',
    'price': 'preco',
    'freight_value': 'frete'
}, inplace=True)

## 5.2 - Visão geral do conjunto de dados

In [ ]:
itens_pedidos

,pedido_id,quantidade,produto_id,vendedor_id,data_limite_envio,preco,frete
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


In [ ]:
print(itens_pedidos.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   pedido_id          112650 non-null  object 
 1   quantidade         112650 non-null  int64  
 2   produto_id         112650 non-null  object 
 3   vendedor_id        112650 non-null  object 
 4   data_limite_envio  112650 non-null  object 
 5   preco              112650 non-null  float64
 6   frete              112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB
None


Valores únicos em cada coluna:

In [ ]:
itens_pedidos.nunique()

pedido_id            98666
quantidade              21
produto_id           32951
vendedor_id           3095
data_limite_envio    93318
preco                 5968
frete                 6999
dtype: int64

Valores duplicados:

In [ ]:
itens_pedidos.duplicated().sum()

0

Valores nulos em cada coluna:

In [ ]:
print(itens_pedidos.isnull().sum())

pedido_id            0
quantidade           0
produto_id           0
vendedor_id          0
data_limite_envio    0
preco                0
frete                0
dtype: int64


## 5.3 - Transformando a coluna 'data_limite_envio' para o formato de data

In [ ]:
itens_pedidos['data_limite_envio'] = pd.to_datetime(itens_pedidos['data_limite_envio'])

## 5.4 - DataFrame itens_pedidos após tratamento

In [ ]:
itens_pedidos.head()

,pedido_id,quantidade,produto_id,vendedor_id,data_limite_envio,preco,frete
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [ ]:
itens_pedidos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   pedido_id          112650 non-null  object        
 1   quantidade         112650 non-null  int64         
 2   produto_id         112650 non-null  object        
 3   vendedor_id        112650 non-null  object        
 4   data_limite_envio  112650 non-null  datetime64[ns]
 5   preco              112650 non-null  float64       
 6   frete              112650 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 6.0+ MB


# <font color=red>6 - Pedidos</font>

## 6.1 - Renomear as colunas

In [ ]:
pedidos.columns.tolist()

['order_id',
 'customer_id',
 'order_status',
 'order_purchase_timestamp',
 'order_approved_at',
 'order_delivered_carrier_date',
 'order_delivered_customer_date',
 'order_estimated_delivery_date']

In [ ]:
mapeamento_colunas = {
    'order_id': 'pedido_id',
    'customer_id': 'cliente_id',
    'order_status': 'pedido_status',
    'order_purchase_timestamp': 'data_compra',
    'order_approved_at': 'data_aprovacao_pedido',
    'order_delivered_carrier_date': 'data_entrega_transportadora',
    'order_delivered_customer_date': 'data_entrega_cliente',
    'order_estimated_delivery_date': 'data_estimada_entrega'
}
pedidos.rename(columns=mapeamento_colunas, inplace=True)

## 6.2 - Visão geral do conjunto de dados

In [ ]:
pedidos.head()

,pedido_id,cliente_id,pedido_status,data_compra,data_aprovacao_pedido,data_entrega_transportadora,data_entrega_cliente,data_estimada_entrega
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [ ]:
print(pedidos.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   pedido_id                    99441 non-null  object
 1   cliente_id                   99441 non-null  object
 2   pedido_status                99441 non-null  object
 3   data_compra                  99441 non-null  object
 4   data_aprovacao_pedido        99281 non-null  object
 5   data_entrega_transportadora  97658 non-null  object
 6   data_entrega_cliente         96476 non-null  object
 7   data_estimada_entrega        99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB
None


Valores únicos em cada coluna:

In [ ]:
pedidos.nunique()

pedido_id                      99441
cliente_id                     99441
pedido_status                      8
data_compra                    98875
data_aprovacao_pedido          90733
data_entrega_transportadora    81018
data_entrega_cliente           95664
data_estimada_entrega            459
dtype: int64

Valores duplicados:

In [ ]:
pedidos.duplicated().sum()

0

Valores nulos em cada coluna:

In [ ]:
pedidos.isnull().sum()

pedido_id                         0
cliente_id                        0
pedido_status                     0
data_compra                       0
data_aprovacao_pedido           160
data_entrega_transportadora    1783
data_entrega_cliente           2965
data_estimada_entrega             0
dtype: int64

## 6.3 - Será removida as linhas irrelevantes, pois serão analisados somente pedidos entregues e concluídos

In [ ]:
pedidos['pedido_status'].value_counts()

pedido_status
delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: count, dtype: int64

Entregue (delivered),
Enviado (shipped),
Cancelado (canceled),
Indisponível (unavailable),
Faturado (invoiced),
Em processamento (processing),
Criado (created),
Aprovado (approved).

Remoção das linhas em que o pedido foi cancelado ou o produto estava indisponível.

In [ ]:
pedidos = pedidos[~pedidos['pedido_status'].isin(['canceled','unavailable'])]

Remoção das linhas em que a data da entrega do pedido está nula e o status do pedido não está como entregue, que indicaria que o pedido ainda não foi entregue.

In [ ]:
pedidos = pedidos[~(pedidos['data_entrega_cliente'].isnull() & (pedidos['pedido_status'] != 'delivered'))]

## 6.4 - Transformar as colunas com data para o formato adequado

In [ ]:
pedidos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 96478 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   pedido_id                    96478 non-null  object
 1   cliente_id                   96478 non-null  object
 2   pedido_status                96478 non-null  object
 3   data_compra                  96478 non-null  object
 4   data_aprovacao_pedido        96464 non-null  object
 5   data_entrega_transportadora  96476 non-null  object
 6   data_entrega_cliente         96470 non-null  object
 7   data_estimada_entrega        96478 non-null  object
dtypes: object(8)
memory usage: 6.6+ MB


In [ ]:
colunas_data = ['data_compra', 'data_aprovacao_pedido', 'data_entrega_transportadora', 'data_entrega_cliente', 'data_estimada_entrega']

for coluna in colunas_data:
    pedidos[coluna] = pd.to_datetime(pedidos[coluna]).dt.floor('s')

## 6.5 - Lidando com valores ausentes

In [ ]:
pedidos.isnull().sum()

pedido_id                       0
cliente_id                      0
pedido_status                   0
data_compra                     0
data_aprovacao_pedido          14
data_entrega_transportadora     2
data_entrega_cliente            8
data_estimada_entrega           0
dtype: int64

### 6.5.1 - Para preencher os dados nulos da coluna "data_aprovacao_pedido" será calculado o tempo médio em que o pedido é aprovado e esse valor será adicionado à data da compra

DataFrame das linhas com dados nulos na coluna 'data_aprovacao_pedido' antes do tratamento.

In [ ]:
linhas_com_nulos = pedidos[pedidos['data_aprovacao_pedido'].isnull()]
pedidos_id_com_nulos = linhas_com_nulos['pedido_id'].tolist()
linhas_com_nulos.head()

,pedido_id,cliente_id,pedido_status,data_compra,data_aprovacao_pedido,data_entrega_transportadora,data_entrega_cliente,data_estimada_entrega
5323,e04abd8149ef81b95221e88f6ed9ab6a,2127dc6603ac33544953ef05ec155771,delivered,2017-02-18 14:40:00,NaT,2017-02-23 12:04:47,2017-03-01 13:25:33,2017-03-17
16567,8a9adc69528e1001fc68dd0aaebbb54a,4c1ccc74e00993733742a3c786dc3c1f,delivered,2017-02-18 12:45:31,NaT,2017-02-23 09:01:52,2017-03-02 10:05:06,2017-03-21
19031,7013bcfc1c97fe719a7b5e05e61c12db,2941af76d38100e0f8740a374f1a5dc3,delivered,2017-02-18 13:29:47,NaT,2017-02-22 16:25:25,2017-03-01 08:07:38,2017-03-17
22663,5cf925b116421afa85ee25e99b4c34fb,29c35fc91fc13fb5073c8f30505d860d,delivered,2017-02-18 16:48:35,NaT,2017-02-22 11:23:10,2017-03-09 07:28:47,2017-03-31
23156,12a95a3c06dbaec84bcfb0e2da5d228a,1e101e0daffaddce8159d25a8e53f2b2,delivered,2017-02-17 13:05:55,NaT,2017-02-22 11:23:11,2017-03-02 11:09:19,2017-03-20


In [ ]:
# Tempo médio de aprovação dos pedidos
tempo_medio_aprovacao = (pedidos['data_aprovacao_pedido'] - pedidos['data_compra']).mean()
# Valor arredondado sem os nanosegundos
tempo_medio_aprovacao = pd.to_timedelta(tempo_medio_aprovacao).floor('1s')
tempo_medio_aprovacao


Timedelta('0 days 10:16:36')

In [ ]:
# Preencher os valores nulos em 'data_aprovacao_pedido' adicionando o tempo médio
pedidos['data_aprovacao_pedido'].fillna(pedidos['data_compra'] + tempo_medio_aprovacao, inplace=True)

DataFrame das linhas com dados nulos na coluna 'data_aprovacao_pedido' após o preenchimento.

In [ ]:
pedido_preenchido= pedidos[pedidos['pedido_id'].isin(pedidos_id_com_nulos)]
pedido_preenchido.head()

,pedido_id,cliente_id,pedido_status,data_compra,data_aprovacao_pedido,data_entrega_transportadora,data_entrega_cliente,data_estimada_entrega
5323,e04abd8149ef81b95221e88f6ed9ab6a,2127dc6603ac33544953ef05ec155771,delivered,2017-02-18 14:40:00,2017-02-19 00:56:36,2017-02-23 12:04:47,2017-03-01 13:25:33,2017-03-17
16567,8a9adc69528e1001fc68dd0aaebbb54a,4c1ccc74e00993733742a3c786dc3c1f,delivered,2017-02-18 12:45:31,2017-02-18 23:02:07,2017-02-23 09:01:52,2017-03-02 10:05:06,2017-03-21
19031,7013bcfc1c97fe719a7b5e05e61c12db,2941af76d38100e0f8740a374f1a5dc3,delivered,2017-02-18 13:29:47,2017-02-18 23:46:23,2017-02-22 16:25:25,2017-03-01 08:07:38,2017-03-17
22663,5cf925b116421afa85ee25e99b4c34fb,29c35fc91fc13fb5073c8f30505d860d,delivered,2017-02-18 16:48:35,2017-02-19 03:05:11,2017-02-22 11:23:10,2017-03-09 07:28:47,2017-03-31
23156,12a95a3c06dbaec84bcfb0e2da5d228a,1e101e0daffaddce8159d25a8e53f2b2,delivered,2017-02-17 13:05:55,2017-02-17 23:22:31,2017-02-22 11:23:11,2017-03-02 11:09:19,2017-03-20


### 6.5.2 - Essa mesma lógica será usada para preencher a coluna 'data_entrega_transportadora' e 'data_entrega_cliente'

In [ ]:
# Tempo médio para entrega na transportadora
tempo_medio_transportadora = (pedidos['data_entrega_transportadora'] - pedidos['data_aprovacao_pedido']).mean()
# Valor arredondado sem os nanosegundos
tempo_medio_transportadora = pd.to_timedelta(tempo_medio_transportadora).floor('1s')
# Preencher os valores nulos em 'data_entrega_transportadora' adicionando o tempo médio
pedidos['data_entrega_transportadora'].fillna(pedidos['data_aprovacao_pedido'] + tempo_medio_transportadora, inplace=True)

In [ ]:
# Tempo médio para entrega no cliente
tempo_medio_cliente = (pedidos['data_entrega_cliente'] - pedidos['data_entrega_transportadora']).mean()
# Valor arredondado sem os nanosegundos
tempo_medio_cliente = pd.to_timedelta(tempo_medio_cliente).floor('1s')
# Preencher os valores nulos em 'data_entrega_cliente' adicionando o tempo médio
pedidos['data_entrega_cliente'].fillna(pedidos['data_entrega_transportadora'] + tempo_medio_cliente, inplace=True)

## 6.6 - Remover a coluna pedido_status porque só ficou o status 'delivered' e por isso se tornou irrelevante

In [ ]:
pedidos.nunique()

pedido_id                      96478
cliente_id                     96478
pedido_status                      1
data_compra                    95956
data_aprovacao_pedido          88288
data_entrega_transportadora    80108
data_entrega_cliente           95665
data_estimada_entrega            445
dtype: int64

In [ ]:
pedidos = pedidos.drop(columns=['pedido_status'])

## 6.7 - DataFrame pedidos após tratamento

In [ ]:
pedidos.head()

,pedido_id,cliente_id,data_compra,data_aprovacao_pedido,data_entrega_transportadora,data_entrega_cliente,data_estimada_entrega
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26


In [ ]:
pedidos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 96478 entries, 0 to 99440
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   pedido_id                    96478 non-null  object        
 1   cliente_id                   96478 non-null  object        
 2   data_compra                  96478 non-null  datetime64[ns]
 3   data_aprovacao_pedido        96478 non-null  datetime64[ns]
 4   data_entrega_transportadora  96478 non-null  datetime64[ns]
 5   data_entrega_cliente         96478 non-null  datetime64[ns]
 6   data_estimada_entrega        96478 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(2)
memory usage: 5.9+ MB


In [ ]:
pedidos.describe()

,data_compra,data_aprovacao_pedido,data_entrega_transportadora,data_entrega_cliente,data_estimada_entrega
count,96478,96478,96478,96478,96478
mean,2018-01-01 23:29:31.939913984,2018-01-02 09:46:08.301104896,2018-01-05 04:57:07.508323328,2018-01-14 12:53:21.279120640,2018-01-25 17:09:52.325711616
min,2016-09-15 12:16:38,2016-09-15 12:16:38,2016-10-08 10:34:01,2016-10-11 13:46:32,2016-10-04 00:00:00
25%,2017-09-14 09:00:23.249999872,2017-09-14 13:30:21.249999872,2017-09-18 16:44:13.750000128,2017-09-25 22:18:14.500000,2017-10-05 00:00:00
50%,2018-01-20 19:45:45,2018-01-22 13:48:43.500000,2018-01-24 16:10:58,2018-02-02 19:41:06,2018-02-16 00:00:00
75%,2018-05-05 18:54:47,2018-05-06 10:52:59.249999872,2018-05-08 14:34:45,2018-05-15 23:09:25.750000128,2018-05-28 00:00:00
max,2018-08-29 15:00:37,2018-08-29 15:10:26,2018-09-11 19:48:28,2018-10-17 13:22:46,2018-10-25 00:00:00


# <font color=red>7 - Pagamentos</font>

## 7.1 - Renomear colunas

In [ ]:
pagamentos.columns.tolist()

['order_id',
 'payment_sequential',
 'payment_type',
 'payment_installments',
 'payment_value']

In [ ]:
mapeamento_colunas = {
    'order_id': 'pedido_id',
    'payment_sequential': 'pagamento_sequencia',
    'payment_type': 'pagamento_forma',
    'payment_installments': 'pagamento_parcelas',
    'payment_value': 'pagamento_valor'
}
pagamentos.rename(columns=mapeamento_colunas, inplace=True)

## 7.2 - Visão geral do conjunto de dados

In [ ]:
pagamentos.head()

,pedido_id,pagamento_sequencia,pagamento_forma,pagamento_parcelas,pagamento_valor
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [ ]:
pagamentos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   pedido_id            103886 non-null  object 
 1   pagamento_sequencia  103886 non-null  int64  
 2   pagamento_forma      103886 non-null  object 
 3   pagamento_parcelas   103886 non-null  int64  
 4   pagamento_valor      103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


Valores únicos em cada coluna:

In [ ]:
pagamentos.nunique()

pedido_id              99440
pagamento_sequencia       29
pagamento_forma            5
pagamento_parcelas        24
pagamento_valor        29077
dtype: int64

Valores duplicados:

In [ ]:
pagamentos.duplicated().sum()

0

Valores nulos em cada coluna:

In [ ]:
print(pagamentos.isnull().sum())

pedido_id              0
pagamento_sequencia    0
pagamento_forma        0
pagamento_parcelas     0
pagamento_valor        0
dtype: int64


## 7.3 - Será removida as linhas com o valor do pagamento igual a 0

Será apagado as 9 linhas em que o valor é 0

In [ ]:
mascara = pagamentos['pagamento_valor'] == 0.0
pagamentos = pagamentos[~mascara]

## 7.4 - Codificação da Variável Categórica 'pagamento_forma' em Numéricos

In [ ]:
formas = pagamentos['pagamento_forma']
lencoder = LabelEncoder()
formas_numeros = lencoder.fit_transform(formas)
pagamentos['pagamento_forma_ref'] = formas_numeros

<ipython-input-120-9be6fa8ea224>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pagamentos['pagamento_forma_ref'] = formas_numeros


## 7.5 - DataFrame pagamentos após tratamento

In [ ]:
pagamentos.head()

,pedido_id,pagamento_sequencia,pagamento_forma,pagamento_parcelas,pagamento_valor,pagamento_forma_ref
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33,1
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39,1
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71,1
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78,1
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45,1


In [ ]:
pagamentos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103877 entries, 0 to 103885
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   pedido_id            103877 non-null  object 
 1   pagamento_sequencia  103877 non-null  int64  
 2   pagamento_forma      103877 non-null  object 
 3   pagamento_parcelas   103877 non-null  int64  
 4   pagamento_valor      103877 non-null  float64
 5   pagamento_forma_ref  103877 non-null  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 5.5+ MB


# <font color=red>8 - Avaliações</font>

In [ ]:
avaliacoes = pd.read_csv('/content/olist_order_reviews_dataset.csv')

## 8.1 - Renomear colunas

In [ ]:
avaliacoes.columns.tolist()

['review_id',
 'order_id',
 'review_score',
 'review_comment_title',
 'review_comment_message',
 'review_creation_date',
 'review_answer_timestamp']

In [ ]:
mapeamento_colunas = {
    'review_id': 'avaliacao_id',
    'order_id': 'pedido_id',
    'review_score': 'avaliacao_pontuacao',
    'review_comment_title': 'avaliacao_titulo',
    'review_comment_message': 'avaliacao_comentario',
    'review_creation_date': 'avaliacao_data',
    'review_answer_timestamp': 'avaliacao_data_resposta'
}
avaliacoes.rename(columns=mapeamento_colunas, inplace=True)

## 8.2 - Visão geral do conjunto de dados

In [ ]:
avaliacoes.head()

,avaliacao_id,pedido_id,avaliacao_pontuacao,avaliacao_titulo,avaliacao_comentario,avaliacao_data,avaliacao_data_resposta
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [ ]:
avaliacoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   avaliacao_id             99224 non-null  object
 1   pedido_id                99224 non-null  object
 2   avaliacao_pontuacao      99224 non-null  int64 
 3   avaliacao_titulo         11568 non-null  object
 4   avaliacao_comentario     40977 non-null  object
 5   avaliacao_data           99224 non-null  object
 6   avaliacao_data_resposta  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [ ]:
avaliacoes.describe()

,avaliacao_pontuacao
count,99224.000000
mean,4.086421
std,1.347579
min,1.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


Valores únicos em cada coluna:

In [ ]:
avaliacoes.nunique()

avaliacao_id               98410
pedido_id                  98673
avaliacao_pontuacao            5
avaliacao_titulo            4527
avaliacao_comentario       36159
avaliacao_data               636
avaliacao_data_resposta    98248
dtype: int64

Valores duplicados:

In [ ]:
avaliacoes.duplicated().sum()

0

Valores nulos em cada coluna:

In [ ]:
print(avaliacoes.isnull().sum())

avaliacao_id                   0
pedido_id                      0
avaliacao_pontuacao            0
avaliacao_titulo           87656
avaliacao_comentario       58247
avaliacao_data                 0
avaliacao_data_resposta        0
dtype: int64


No caso das avaliaões sem título e sem comentários será deixado com os valores nulos, primeiro porque condiz com a realidade a qual o cliente não quis colocar nenhum comentário, também porque não afetará a análise de sentimentos e não será usado em modelos de machine learning.

## 8.3 - Transformar as colunas com data para o formato adequado

In [ ]:
colunas_data = ['avaliacao_data', 'avaliacao_data_resposta']

for coluna in colunas_data:
    avaliacoes[coluna] = pd.to_datetime(avaliacoes[coluna])

## 8.4 - DataFrame avaliações após tratamento

In [ ]:
avaliacoes.head()

,avaliacao_id,pedido_id,avaliacao_pontuacao,avaliacao_titulo,avaliacao_comentario,avaliacao_data,avaliacao_data_resposta
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01,2018-03-02 10:26:53


In [ ]:
avaliacoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   avaliacao_id             99224 non-null  object        
 1   pedido_id                99224 non-null  object        
 2   avaliacao_pontuacao      99224 non-null  int64         
 3   avaliacao_titulo         11568 non-null  object        
 4   avaliacao_comentario     40977 non-null  object        
 5   avaliacao_data           99224 non-null  datetime64[ns]
 6   avaliacao_data_resposta  99224 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 5.3+ MB


# <font color=red>9 - Junção dos DataFrames para ser usado nas próximas análises e na mineração de dados</font>

In [ ]:
relatorio_vendas = itens_pedidos.merge(pedidos, on='pedido_id') \
                    .merge(pagamentos, on=['pedido_id']) \
                    .merge(avaliacoes, on='pedido_id') \
                    .merge(produtos, on='produto_id') \
                    .merge(clientes, on='cliente_id') \
                    .merge(vendedores, on='vendedor_id')
relatorio_vendas.head()

,pedido_id,quantidade,produto_id,vendedor_id,data_limite_envio,preco,frete,cliente_id,data_compra,data_aprovacao_pedido,...,cliente_lon,vendedor_cep,vendedor_cidade,vendedor_cidade_ref,vendedor_estado,vendedor_estado_ref,vendedor_regiao,vendedor_regiao_ref,vendedor_lat,vendedor_lon
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29,3ce436f183e68e07877b285a838db11a,2017-09-13 08:59:02,2017-09-13 09:45:35,...,-41.316134,27277,volta redonda,4173,SP,26,sudeste,3,-22.499325,-44.125997
1,130898c0987d1801452a8ed92a670612,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-07-05 02:44:11,55.9,17.96,e6eecc5a77de221464d1c4eaff0a9b64,2017-06-28 11:52:20,2017-06-29 02:44:11,...,-51.717782,27277,volta redonda,4173,SP,26,sudeste,3,-22.499325,-44.125997
2,532ed5e14e24ae1f0d735b91524b98b9,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2018-05-23 10:56:25,64.9,18.33,4ef55bf80f711b372afebcb7c715344a,2018-05-18 10:25:53,2018-05-18 12:31:43,...,-43.973181,27277,volta redonda,4173,SP,26,sudeste,3,-22.499325,-44.125997
3,6f8c31653edb8c83e1a739408b5ff750,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-08-07 18:55:08,58.9,16.17,30407a72ad8b3f4df4d15369126b20c9,2017-08-01 18:38:42,2017-08-01 18:55:08,...,-49.126570,27277,volta redonda,4173,SP,26,sudeste,3,-22.499325,-44.125997
4,7d19f4ef4d04461989632411b7e588b9,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-08-16 22:05:11,58.9,13.29,91a792fef70ecd8cc69d3c7feb3d12da,2017-08-10 21:48:40,2017-08-10 22:05:11,...,-43.794100,27277,volta redonda,4173,SP,26,sudeste,3,-22.499325,-44.125997


In [ ]:
relatorio_vendas.shape

(114855, 46)

In [ ]:
relatorio_vendas.to_csv('relatorio_vendas.csv', index=False)

In [ ]:
relatorio_vendas.isna().sum()

pedido_id                           0
quantidade                          0
produto_id                          0
vendedor_id                         0
data_limite_envio                   0
preco                               0
frete                               0
cliente_id                          0
data_compra                         0
data_aprovacao_pedido               0
data_entrega_transportadora         0
data_entrega_cliente                0
data_estimada_entrega               0
pagamento_sequencia                 0
pagamento_forma                     0
pagamento_parcelas                  0
pagamento_valor                     0
pagamento_forma_ref                 0
avaliacao_id                        0
avaliacao_pontuacao                 0
avaliacao_titulo               101271
avaliacao_comentario            66764
avaliacao_data                      0
avaliacao_data_resposta             0
produto_categoria                   0
produto                             0
produto_cate

In [ ]:
relatorio_vendas.describe()

,quantidade,data_limite_envio,preco,frete,data_compra,data_aprovacao_pedido,data_entrega_transportadora,data_entrega_cliente,data_estimada_entrega,pagamento_sequencia,...,cliente_estado_ref,cliente_regiao_ref,cliente_lat,cliente_lon,vendedor_cep,vendedor_cidade_ref,vendedor_estado_ref,vendedor_regiao_ref,vendedor_lat,vendedor_lon
count,114855.000000,114855,114855.000000,114855.000000,114855,114855,114855,114855,114855,114855.000000,...,114855.000000,114855.000000,114855.000000,114855.000000,114855.000000,114855.000000,114855.000000,114855.000000,114855.000000,114855.000000
mean,1.194593,2018-01-07 04:24:27.510234368,119.775293,19.974419,2017-12-31 13:48:49.678472704,2018-01-01 00:14:40.536720384,2018-01-03 20:29:44.335179264,2018-01-13 00:28:53.661259776,2018-01-24 09:45:30.207653120,1.091472,...,19.696809,2.765766,-21.234846,-46.210134,24461.331392,2546.782935,23.219721,3.058099,-22.794561,-47.183752
min,1.000000,2016-10-08 10:34:01,0.850000,0.000000,2016-10-03 09:44:50,2016-10-04 09:43:32,2016-10-08 10:34:01,2016-10-11 13:46:32,2016-10-27 00:00:00,1.000000,...,1.000000,0.000000,-33.691550,-72.664409,1001.000000,1.000000,3.000000,0.000000,-32.095725,-63.895935
25%,1.000000,2017-09-19 21:05:46,39.900000,13.080000,2017-09-12 19:43:19.500000,2017-09-13 08:24:41.500000,2017-09-15 14:36:13,2017-09-23 12:01:26.500000,2017-10-04 00:00:00,1.000000,...,13.000000,3.000000,-23.591311,-48.106492,6429.000000,1627.000000,23.000000,3.000000,-23.607049,-48.838906
50%,1.000000,2018-01-25 19:20:37,74.900000,16.280000,2018-01-19 03:30:43,2018-01-19 14:19:24,2018-01-23 19:28:12,2018-02-01 16:53:10,2018-02-15 00:00:00,1.000000,...,23.000000,3.000000,-22.926903,-46.633693,13660.000000,2984.000000,26.000000,3.000000,-23.421243,-46.745476
75%,1.000000,2018-05-10 12:29:44,132.900000,21.150000,2018-05-04 14:22:16,2018-05-04 21:34:25,2018-05-08 08:42:30,2018-05-15 14:14:13,2018-05-25 00:00:00,1.000000,...,26.000000,3.000000,-20.156348,-43.656152,28035.000000,3658.000000,26.000000,3.000000,-21.754867,-46.521174
max,21.000000,2020-04-09 22:35:08,6735.000000,409.680000,2018-08-29 15:00:37,2018-08-29 15:10:26,2018-09-11 19:48:28,2018-10-17 13:22:46,2018-10-25 00:00:00,26.000000,...,27.000000,4.000000,42.184003,-4.947823,99730.000000,4188.000000,26.000000,4.000000,-2.505689,-15.826959
std,0.685727,NaN,181.426950,15.703408,NaN,NaN,NaN,NaN,NaN,0.688680,...,7.065804,1.000598,5.563170,4.046570,27594.924211,1158.557985,5.216230,0.567038,2.697265,2.632427


In [ ]:
# Especifique o formato desejado
formato_desejado = '%Y-%m-%d %H:%M:%S'

# Crie uma lista para armazenar datas fora do formato
datas_fora_do_formato = []

colunas_de_data = ['data_limite_envio', 'data_compra', 'data_aprovacao_pedido', 'data_entrega_transportadora', 'data_entrega_cliente', 'data_estimada_entrega', 'avaliacao_data', 'avaliacao_data_resposta']


# Itere pelas colunas de data
for coluna in colunas_de_data:
    for idx, data in enumerate(relatorio_vendas[coluna]):
        try:
            # Tente converter a data para o formato desejado
            pd.to_datetime(data, format=formato_desejado)
        except ValueError:
            # Se houver uma exceção, a data está fora do formato
            datas_fora_do_formato.append((coluna, idx, data))

# Verifique se há datas fora do formato
if not datas_fora_do_formato:
    print("Todas as datas estão no formato especificado.")
else:
    # Exiba as datas fora do formato
    for coluna, idx, data in datas_fora_do_formato:
        print(f"Coluna: {coluna}, Índice {idx}: Data fora do formato - {data}")

Todas as datas estão no formato especificado.
